基本示例

```python
for chunk in graph.stream(inputs, stream_mode="updates"):
    print(chunk)
```

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

class State(TypedDict):
    topic: str
    joke: str

def refine_topic(state: State):
    return {"topic": state["topic"] + " and cats"}

def generate_joke(state: State):
    return {"joke": f"This is a joke about {state['topic']}"}

graph = (
    StateGraph(State)
    .add_node(refine_topic)
    .add_node(generate_joke)
    .add_edge(START, "refine_topic")
    .add_edge("refine_topic", "generate_joke")
    .add_edge("generate_joke", END)
    .compile()
)

In [2]:
# The stream() method returns an iterator that yields streamed outputs
for chunk in graph.stream(  
    {"topic": "ice cream"},
    # Set stream_mode="updates" to stream only the updates to the graph state after each node
    # Other stream modes are also available. See supported stream modes for details
    stream_mode="updates",  
):
    print(chunk)

{'refine_topic': {'topic': 'ice cream and cats'}}
{'generate_joke': {'joke': 'This is a joke about ice cream and cats'}}


In [ ]:
for chunk in graph.stream(  
    {"topic": "ice cream"},
    stream_mode=["updates", "custom"],  
):
    print(chunk)

('updates', {'refine_topic': {'topic': 'ice cream and cats'}})
('updates', {'generate_joke': {'joke': 'This is a joke about ice cream and cats'}})


In [6]:
for chunk in graph.stream(  
    {"topic": "ice cream"},
    stream_mode=["debug"],  
):
    print(chunk)

('debug', {'step': 1, 'timestamp': '2025-11-21T07:04:21.856210+00:00', 'type': 'task', 'payload': {'id': '7489b9c4-babf-ce85-4ec4-5cab47a7403f', 'name': 'refine_topic', 'input': {'topic': 'ice cream'}, 'triggers': ('branch:to:refine_topic',)}})
('debug', {'step': 1, 'timestamp': '2025-11-21T07:04:21.856489+00:00', 'type': 'task_result', 'payload': {'id': '7489b9c4-babf-ce85-4ec4-5cab47a7403f', 'name': 'refine_topic', 'error': None, 'result': {'topic': 'ice cream and cats'}, 'interrupts': []}})
('debug', {'step': 2, 'timestamp': '2025-11-21T07:04:21.856598+00:00', 'type': 'task', 'payload': {'id': '3006ea95-9265-ce97-270d-ca85adabd036', 'name': 'generate_joke', 'input': {'topic': 'ice cream and cats'}, 'triggers': ('branch:to:generate_joke',)}})
('debug', {'step': 2, 'timestamp': '2025-11-21T07:04:21.856709+00:00', 'type': 'task_result', 'payload': {'id': '3006ea95-9265-ce97-270d-ca85adabd036', 'name': 'generate_joke', 'error': None, 'result': {'joke': 'This is a joke about ice cream an